# Using beamforming to localize oscillatory power modulations

The aim of this section is to localize the modulations of oscillatory power. This will be done using the Dynamical Imaging of Coherent Sources (DICS) approach (Gross et al, 2001). This is a beamforming methodology based on spatial filtering operating in the frequency domain. For each grid point in the brain, a spatial filter is derived based on the forward model and the cross-spectral density (CSD) matrix. A multitaper approach is applied for the spectral estimates.
The approach allows for estimating the power modulations at each grid points which then is mapped onto the indiviual structural MRIs. Prewhitening of the data allows for combining the information from the gradiometers and the magnetomers.


## Preparation

Import the relevant Python modules:

In [38]:
# mount the drive (you'll have to click through the popup to allow it)
# google colab friendship block

from IPython.display import clear_output

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#!pip install openneuro-py
#!pip install --upgrade mne-bids[full]
#!pip install ipympl  # for the widgets in ipynb
!pip install install ipywidgets

clear_output()  # just clear everything because the output is gross and unnecessary

In [3]:
import os
import os.path as op
import numpy as np

import mne
from mne_bids import BIDSPath
from mne.cov import compute_covariance
from mne.beamformer import make_dics, apply_dics_csd
from mne.time_frequency import csd_multitaper

### File overview
The chapter relies on the input file (embedded in the BIDS structure):
~~~
<BIDS_ROOT>/derivatives/Analysis/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-12_epo.fif

<BIDS_ROOT>/derivatives/Analysis/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-12_src.fif
<BIDS_ROOT>/derivatives/FreeSurfer
<BIDS_ROOT>/derivatives/Analysis/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-12_fwd.fif


Set the paths and name for the data. Note that these will depend on your local setup.

In [35]:
subject = 'CB069'
session = '1'
task = 'dur'  # name of the task
run = '00'
meg_suffix = 'meg'
epo_suffix = 'epo'
mri_suffix = 'T1w'
bem_suffix = 'bem-sol'
src_suffix = 'src'
fwd_suffix = 'fwd'

bids_root = 'drive/MyDrive/GripS MEG/Data/COGITATE'
deriv_root = 'drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis'

bids_path = BIDSPath(subject=subject, session=session, task=task, run=run, suffix=epo_suffix, datatype='meg', root=deriv_root, extension='.fif', check=False)


fwd_fname = bids_path.basename.replace(epo_suffix, fwd_suffix)
fwd_file = op.join(bids_path.directory, fwd_fname)
src_file = fwd_file.replace(fwd_suffix, src_suffix)
bem_file = fwd_file.replace(fwd_suffix, bem_suffix)


fs_subject = 'sub-CB069_ses-1'
fs_root = op.join(bids_root, 'derivatives', 'FreeSurfer')

print(bids_path.basename,bids_path.fpath)
print(src_file)
print(fs_root)
print(fwd_file)



sub-CB069_ses-1_task-dur_run-00_epo.fif drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis/sub-CB069/ses-1/meg/sub-CB069_ses-1_task-dur_run-00_epo.fif
drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis/sub-CB069/ses-1/meg/sub-CB069_ses-1_task-dur_run-00_src.fif
drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/FreeSurfer
drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis/sub-CB069/ses-1/meg/sub-CB069_ses-1_task-dur_run-00_fwd.fif


## The forward model

In this example we will use the forward model calculated in the previous section 'Head model and forward computations'. The forward model is constructed for the 306 MEG sensors (both magnetometer and gradiometers) with respect to the volumetric forward solution (not the brain surface solution).

In [24]:
fwd = mne.read_forward_solution(fwd_file)

Reading forward solution from /content/drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis/sub-CB069/ses-1/meg/sub-CB069_ses-1_task-dur_run-00_fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (516 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


## Reading the trials

Now read the epochs for all trial types:

In [25]:
epochs = mne.read_epochs(bids_path.fpath, proj=False)

Reading /content/drive/MyDrive/GripS MEG/Data/COGITATE/derivatives/analysis/sub-CB069/ses-1/meg/sub-CB069_ses-1_task-dur_run-00_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
280 matching events found
No baseline correction applied


## Source modeling of modulations of alpha band activity

### Calculating the cross-spectral density matrices for the alpha band

The spatial filters in the DICS beamforming approach are calculated using the forward model and the cross-spectral density (CSD) matrices. We here focus on the -0.8 to -0.3 s interval prior to the onset of the gratings and the 0.3 to 0.8 s interval when the gratings are presented. During the presentation interval, the alpha band activity is suppressed (see the section Time-Frequency Representations of Power).

We start by calculating the CSDs for the 8 to 12 Hz alpha band for all conditions and the pre- and post-interval. Since we are using the multitaper approach we are defining the bandwidth to 3 Hz. This results in one taper (as N < 2 ΔF ΔT = 2 0.5).  

In [26]:
csd_aleft_pre   = csd_multitaper(epochs['face'],  fmin=8, fmax=12, tmin=-0.2, tmax=0.1, bandwidth = 5, low_bias = True, verbose = False, n_jobs=-1)
csd_aright_pre  = csd_multitaper(epochs['object'], fmin=8, fmax=12, tmin=-0.2, tmax=0.1, bandwidth = 5, low_bias = True,verbose = False, n_jobs=-1)

csd_aleft_cue =  csd_multitaper(epochs['face'],  fmin=8, fmax=12, tmin=-0.1, tmax=0.1, bandwidth = 5,  low_bias = True, verbose = False, n_jobs=-1)
csd_aleft_act =  csd_multitaper(epochs['face'],  fmin=8, fmax=12, tmin= 0.1, tmax= 0.5, bandwidth = 5, low_bias = True, verbose = False, n_jobs=-1)
csd_aright_cue = csd_multitaper(epochs['object'], fmin=8, fmax=12, tmin=-0.1, tmax=0.1, bandwidth = 5,  low_bias = True, verbose = False, n_jobs=-1)
csd_aright_act = csd_multitaper(epochs['object'], fmin=8, fmax=12, tmin= 0.1, tmax= 0.5, bandwidth = 5, low_bias = True, verbose = False, n_jobs=-1)

<ipython-input-26-b49fc7b2d34f>:4: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  csd_aleft_cue =  csd_multitaper(epochs['face'],  fmin=8, fmax=12, tmin=-0.1, tmax=0.1, bandwidth = 5,  low_bias = True, verbose = False, n_jobs=-1)
<ipython-input-26-b49fc7b2d34f>:6: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  csd_aright_cue = csd_multitaper(epochs['object'], fmin=8, fmax=12, tmin=-0.1, tmax=0.1, bandwidth = 5,  low_bias = True, verbose = False, n_jobs=-1)


The spatial filters must be based on the covariance matrices combined ('common spatial filter'). Therefore the CSDs are averaged:

In [27]:
csd_acommon = csd_aleft_cue
csd_anoise =  csd_aleft_pre
csd_acommon._data = (csd_aleft_cue._data + csd_aright_cue._data + csd_aleft_act._data + csd_aright_act._data)/4

csd_anoise._data = (csd_aleft_pre._data + csd_aright_pre._data)/2

For calculating the spatial filters we also need to derive the rank of the CSD. It will be similar to the the rank of the coveriance matrix:

In [28]:
rank = mne.compute_rank(epochs, tol=1e-6, tol_kind='relative', proj = False)
print(rank)

Computing rank from data with rank=None
    Estimated rank (mag + grad): 68
Found multiple SSS records. Using the first.
    MEG: rank 68 computed from 306 data channels with 0 projectors
{'meg': 68}


In [29]:
rank = {'meg': 68}

Note that the rank is 69 eventhough there were 306 used in the calculations. The rank deficiency is caused by the SSS and the ICA algorithms and therefore requires a regulatization before inverting in the beamforming calculation.

### Derive the spatial filters for the alpha band

Now calculate the filters using the forward model `fwd` and the combined CSD matrix `csd_common`. The filters are regularized corresponding to loading the diagonal of the covariance matrix with 5% of the sensor power. For each source the orientation is optimized to maximize the power of the output. Defining *noise_csd* serves to perform spatial prewhitening of the data (Sekihara et al. 2008, Engemann and Gramfort 2015, Westner et al. 2021). We here do this using data from the pre-stimulus interval; however, empty room data could be used as well. As a result the magnetometer and gradiometer signals are uncorrelated to each other but have the same variance. Since the whitening is done separately for the magnetometers and gradiometers, the scale difference between the scale disappears allowing for the data from the two sensor types to be combined. The compensentation for depth bias is set `depth = 0` as condition or intervals will be compared. The relative comparisons alleviate depth bias. The `mean` in `csd_common.mean()` serves to average the CSDs over frequencies. Given that the covariance (or CSD) are severely rank-deficient due to the application of SSS and ICA, MNE-Python automatically computes a truncated pseudo-inverse using the rank calculated above. For the data ($X$) with rank $k$ the truncated pseudo-inverse ($X^\dagger$) is calculated using singular value decomposition $ X = U \Sigma V^T$ ; $X^\dagger = V \Sigma^{-1}_k U^T$
where we consider the first $k$ diagonal values of $\Sigma$ as well as the first $k$ columns of $U$ and $V$.  This allows that the DISC algorithm is effectively applied in a virtual channel space of size `rank` (= 69 in this case) and no further regularization is required (`reg = 0`).

In [30]:
afilters = make_dics(epochs.info, fwd, csd_acommon.mean() , noise_csd=csd_anoise.mean(),
                    reg=0, pick_ori='max-power', reduce_rank=True, real_filter=True, rank=rank, depth = 0)

Identifying common channels ...
Identifying common channels ...
Computing inverse operator with 306 channels.
    306 out of 306 channels remain after picking
Selected 306 channels
Whitening the forward solution.
Computing rank from covariance with rank={'meg': 68}
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing rank from covariance with rank={'meg': 68}
Computing rank from covariance with rank={'meg': 68}
Computing DICS spatial filters...
Computing beamformer filters for 516 sources
Filter computation complete


In [31]:
stc_aleft_cue, freqs = apply_dics_csd(csd_aleft_cue.mean(), afilters)
stc_aleft_act, freqs  = apply_dics_csd(csd_aleft_act.mean(), afilters)
stc_aright_cue, freqs = apply_dics_csd(csd_aright_cue.mean(), afilters)
stc_aright_act, freqs  = apply_dics_csd(csd_aright_act.mean(), afilters)

Computing DICS source power...
[done]
Computing DICS source power...
[done]
Computing DICS source power...
[done]
Computing DICS source power...
[done]


Calculate the relative change in power; i.e. the change in alpha power comparing the interval before and after the moving grating onset. Note that we are always comparing intervals or conditions in order to remove the depth bias confound:

In [32]:
stc_arel = ((stc_aleft_act + stc_aright_act) - (stc_aleft_cue + stc_aright_cue)) / (stc_aleft_cue + stc_aright_cue)

Calculate the change in relative power for comparing the right versus left attention conditions during the moving gratings interval:

In [33]:
stc_aRvsL = (stc_aright_cue - stc_aleft_cue) / (stc_aright_cue + stc_aleft_cue)
stc_aRvsL = (stc_aright_act - stc_aleft_act) / (stc_aright_act + stc_aleft_act)

### Plotting the alpha band modulation


Now plot the source results:

In [39]:
src = fwd['src']
print(src)

%matplotlib inline
stc_arel.plot(src=src, subject=fs_subject, subjects_dir=fs_root, verbose = True);


<SourceSpaces: [<surface (lh), n_vertices=155732, n_used=258>, <surface (rh), n_vertices=155772, n_used=258>] head coords, subject 'sub-CB069_ses-1', ~28.6 MB>


RuntimeError: Could not load any valid 3D backend
pyvistaqt: No module named 'qtpy'
notebook: No module named 'pyvista'

 install pyvistaqt, using pip or conda:
'pip install pyvistaqt'
'conda install -c conda-forge pyvistaqt'

 or install ipywidgets, if using a notebook backend
'pip install ipywidgets'
'conda install -c conda-forge ipywidgets'

The above plot show the relative depression in the alpha following the onset of moving gratings. The blue island reflects the occipital depression in alpha power.

To browse the slice use the following command and the mouse to navigate:

In [37]:
%matplotlib inline
stc_aRvsL.plot(src=src, subject=fs_subject, subjects_dir=fs_root, mode='stat_map');

TypeError: _BaseSourceEstimate.plot() got an unexpected keyword argument 'mode'

The sources in the posterior areas reflect the modulation of the alpha band activity when the partipant is attending to the right versus left moving gratings. The right orange island reflects the well-known ipsi-lateral reflective increase in alpha band power.  

**Question 1** Plot the DISC output (of e.g. *stc_aright_act*) on the MRI prior to doing a comparision. Interpret the outcome.

**Question 2** Change the rank of the data when calculating the filter to 100: `rank={'meg':100}`. How does it affect the outcome?


# Source modeling of modulations of gamma band activity

We will next localize the sources associated with the modulation in the 60 - 90 Hz gamma band. For this, we will apply the multitaper approach to ensure suffient spectral smoothing.

As a first step, calculate the CSD for the pre- and post-interval as well as all conditions. The bandwidth here is 2ΔF = 8 Hz and the time window ΔT = 0.475 s. This results in 3 tapers (as N <  2ΔF ΔT).

In [ ]:
print('Pre')
csd_gleft_pre   = csd_multitaper(epochs['cue_left'],  fmin=60, fmax=90, tmin=-1.9, tmax=-1.4, bandwidth = 8, adaptive=True, low_bias = True,verbose = False, n_jobs=4)
csd_gright_pre  = csd_multitaper(epochs['cue_right'], fmin=60, fmax=90, tmin=-1.9, tmax=-1.4, bandwidth = 8, adaptive=True, low_bias = True,verbose = False, n_jobs=4)

print('Cue')
csd_gleft_cue =  csd_multitaper(epochs['cue_left'],  fmin=60, fmax=90, tmin=-0.6, tmax=-0.1, bandwidth = 8, adaptive=True, low_bias = True, verbose = False, n_jobs=4)
csd_gright_cue = csd_multitaper(epochs['cue_right'], fmin=60, fmax=90, tmin=-0.6, tmax=-0.1, bandwidth = 8, adaptive=True, low_bias = True, verbose = False, n_jobs=4)

print('Stim')
csd_gleft_act =  csd_multitaper(epochs['cue_left'],  fmin=60, fmax=90, tmin= 0.1, tmax= 0.6, bandwidth = 8, adaptive=True, low_bias = True, verbose = False, n_jobs=4)
csd_gright_act = csd_multitaper(epochs['cue_right'], fmin=60, fmax=90, tmin= 0.1, tmax= 0.6, bandwidth = 8, adaptive=True, low_bias = True, verbose = False, n_jobs=4)


In [ ]:
csd_gcommon = csd_gleft_cue
csd_gnoise =  csd_face_pre

csd_gcommon._data = (csd_gleft_cue._data + csd_gright_cue._data + csd_gleft_act._data + csd_gright_act._data)/4
csd_gnoise._data = (csd_gleft_pre._data + csd_gright_pre._data)/2

### Derive the spatial filters for the gamma band

The filters will be calculated on the basis of the common CSD. The rank estimate based on the covariance matrix will be used. The deduced rank is due to the SSS as well as the ICA.


In [ ]:
gfilters = make_dics(epochs.info, fwd, csd_gcommon.mean() , noise_csd=csd_gnoise.mean(), reg=0, pick_ori='max-power',
                        reduce_rank=True, real_filter=True, rank=rank, depth = 0)


### Apply the filters and plot the gamma modulation

The filters are then applied:

In [ ]:
stc_gleft_cue, freqs = apply_dics_csd(csd_gleft_cue.mean(), gfilters)
stc_gleft_act, freqs  = apply_dics_csd(csd_gleft_act.mean(), gfilters)
stc_gright_cue, freqs = apply_dics_csd(csd_gright_cue.mean(), gfilters)
stc_gright_act, freqs  = apply_dics_csd(csd_gright_act.mean(), gfilters)

Now calculate the relative modulation in gamma power with respect to the baseline:

In [ ]:
stc_grel = ((stc_gleft_act + stc_gright_act) - (stc_gleft_cue + stc_gright_cue)) / (stc_gleft_cue + stc_gright_cue)

and subsequently the hemisperic lateralization with respect to attention:

In [ ]:
stc_gRvsL = (stc_gright_act - stc_gleft_act) / (stc_gright_act + stc_gleft_act)

and plot the relative modulation in the interactive plot. Click on the visual cortex and note the increase in gamma power.

In [ ]:
src = fwd['src']
%matplotlib qt

brain = stc_grel.plot(src=src, subject=fs_subject, subjects_dir=fs_root, mode='stat_map',clim=dict(kind='value', lims=[0.,0.05,0.1]));

Use the navigation to move to visual cortex.
The increase of the gamma activity is correclty localized in early visual cortex along the calcarine sulcus.  

To illustrate the modulation in gamma band activity with attention write:

In [ ]:
%matplotlib qt
stc_gRvsL.plot(src=src, subject=fs_subject, subjects_dir=fs_root,
              mode='stat_map',clim=dict(kind='value', pos_lims=[0.,0.05, 0.1],
                                        neg_lims=[-0.1, -0.05, 0]));

The above plots should be interpreted with caution. The right hemisphere decrease is expected, but it is not complemented by an increase in the left hemisphetre. This result is most likely explained by the SNR being too low.

## Preregistration and publications


Example for text to be used for publications and preregistration:

"The Dynamics Imaging of Coherent Sources (DICSs) approach  (Gross et al., 2001) was applied to localize the modulations of oscillatory brain activity. To localize the modulation in the alpha band, the cross-spectral density (CSD) was calculated for the 8 to 12 Hz band using a multi-taper approach applying 2 Hz spectral smoothing. The CSD matrix was then calculated for the 300 - 800 ms stimulus interval in which the gratings were shown as well as the -800 – -300 ms pre-stimulus interval (using one DPSS taper). The combined CSD was used with the forward model to create a common spatial filter. This was done by first estimating the rank of the data (being reduced to ~70 due to the SSS and ICA approaches). The data were spatially pre-whitened using the covariance matrix from the baseline interval which allowed for combining the gradiometer and magnetometer data in the source modelling. The truncated pseudo-inverse was then calculated based on the estimated rank (Jas et al., 2018). For each source, the orientation was optimized to maximize the power of the output. The spatial filter was then applied to the CSDs to calculate the relative change in alpha power. As similar approach was used for the gamma band now considering the 60 - 90 Hz band and 8 Hz spectral smoothing (resulting in 3 DPSS tapers). The pre-stimulus time-interval was -0.6 to -0.125 s and the post-stimulus interval was 0.125 to 0.6 s. The source data were then mapped to the structural MRI. "

## References

* Gross J, Kujala J, Hamalainen M, Timmermann L, Schnitzler A, Salmelin R. (2001) Dynamic imaging of coherent sources: Studying neural interactions in the human brain. Proc Natl Acad Sci U S A. 98(2):694-9.

* Engemann DA, Gramfort A. (2015) Automated model selection in covariance estimation and spatial whitening of MEG and EEG signals. Neuroimage 108:328-42.

* Sekihara K, Hild KE, Dalal SS, Nagarajan SS. (2008) Performance of prewhitening beamforming in MEG dual experimental conditions. IEEE Trans Biomed Eng. 55(3):1112-21.

* van Vliet M, Liljeström M, Aro S, Salmelin R, Kujala J. (2018) Analysis of Functional Connectivity and Oscillatory Power Using DICS: From Raw MEG Data to Group-Level Statistics in Python. Front Neurosci. 12:586.

* Westner BU, Dalal SS, Gramfort A, Litvak V, Mosher JC, Oostenveld R, Schoffelen JM. (2021) A unified view on beamformers for M/EEG source reconstruction. Neuroimage 246:118789.